In [ ]:
#| default_exp distributed.fugue

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# FugueBackend

> The computational efficiency of `StatsForecast` can be tracked to its two core components:<br>1. Its `models` written in NumBa that optimizes Python code to reach C speeds.<br>2. Its `core.StatsForecast` class that enables distributed computing.<br><br>Here we use [Fugue](https://github.com/fugue-project/fugue) which is a unified interface for `Dask` and `Spark`.<br><br>

In [ ]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| export
import inspect
from typing import Any, Dict, List

import numpy as np
import pandas as pd
from fugue import transform, DataFrame, FugueWorkflow, ExecutionEngine
from fugue.collections.yielded import Yielded
from fugue.constants import FUGUE_CONF_WORKFLOW_EXCEPTION_INJECT
from statsforecast.core import _StatsForecast, ParallelBackend, make_backend
from triad import Schema

/Users/kevinkho/Work/statsforecast/statsforecast/core.py:24: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
#| export
def _cotransform(
    df1: Any,
    df2: Any,
    using: Any,
    schema: Any = None,
    params: Any = None,
    partition: Any = None,
    engine: Any = None,
    engine_conf: Any = None,
    force_output_fugue_dataframe: bool = False,
    as_local: bool = False,
) -> Any:
    dag = FugueWorkflow(compile_conf={FUGUE_CONF_WORKFLOW_EXCEPTION_INJECT: 0})
    
    src = dag.create_data(df1).zip(dag.create_data(df2), partition=partition)
    tdf = src.transform(
        using=using,
        schema=schema,
        params=params,
        pre_partition=partition,
    )
    tdf.yield_dataframe_as("result", as_local=as_local)
    dag.run(engine, conf=engine_conf)
    result = dag.yields["result"].result  # type:ignore
    if force_output_fugue_dataframe or isinstance(df1, (DataFrame, Yielded)):
        return result
    return result.as_pandas() if result.is_local else result.native  # type:ignore

In [ ]:
#| export
class FugueBackend(ParallelBackend):
    """FugueBackend for Distributed Computation.
    [Source code](https://github.com/Nixtla/statsforecast/blob/main/statsforecast/distributed/fugue.py).

    This class uses [Fugue](https://github.com/fugue-project/fugue) backend capable of distributing 
    computation on Spark, Dask and Ray without any rewrites.

    **Parameters:**<br>
    `engine`: fugue.ExecutionEngine, a selection between Spark, Dask, and Ray.<br>
    `conf`: fugue.Config, engine configuration.<br>
    `**transform_kwargs`: additional kwargs for Fugue's transform method.<br>

    **Notes:**<br>
    A short introduction to Fugue, with examples on how to scale pandas code to Spark, Dask or Ray
     is available [here](https://fugue-tutorials.readthedocs.io/tutorials/quick_look/ten_minutes.html).
    """
    def __init__(
            self, 
            engine: Any = None,
            conf: Any = None,
            **transform_kwargs: Any
        ):        
        self._engine = engine
        self._conf = conf
        self._transform_kwargs = dict(transform_kwargs)

    def __getstate__(self) -> Dict[str, Any]:
        return {}

    def forecast(
            self, 
            df,
            models,
            freq,
            fallback_model = None,
            X_df = None,
            **kwargs: Any,
        ) -> Any:
        """Memory Efficient core.StatsForecast predictions with FugueBackend.

        This method uses Fugue's transform function, in combination with 
        `core.StatsForecast`'s forecast to efficiently fit a list of StatsForecast models.

        **Parameters:**<br>
        `df`: pandas.DataFrame, with columns [`unique_id`, `ds`, `y`] and exogenous.<br>
        `freq`: str, frequency of the data, [pandas available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>
        `models`: List[typing.Any], list of instantiated objects `StatsForecast.models`.<br>
        `fallback_model`: Any, Model to be used if a model fails.<br>
        `X_df`: pandas.DataFrame, with [unique_id, ds] columns and df’s future exogenous.
        `**kwargs`: Additional `core.StatsForecast` parameters. Example forecast horizon `h`.<br>

        **Returns:**<br>
        `fcsts_df`: pandas.DataFrame, with `models` columns for point predictions and probabilistic
        predictions for all fitted `models`.<br>
        
        **References:**<br>
        For more information check the 
        [Fugue's transform](https://fugue-tutorials.readthedocs.io/tutorials/beginner/transform.html)
        tutorial.<br>
        The [core.StatsForecast's forecast](https://nixtla.github.io/statsforecast/core.html#statsforecast.forecast)
        method documentation.<br>
        Or the list of available [StatsForecast's models](https://nixtla.github.io/statsforecast/src/core/models.html).
        """
        level = kwargs.get("level", [])
        schema = "*-y+" + str(self._get_output_schema(models, level))
        if X_df is None:
            return transform(
                df,
                self._forecast_series,
                params=dict(models=models, freq=freq, 
                            kwargs=kwargs, fallback_model=fallback_model),
                schema=schema,
                partition={"by": "unique_id"},
                engine=self._engine,
                engine_conf=self._conf,
                **self._transform_kwargs,
            )
        else:
            schema = "unique_id:str,ds:str," + str(self._get_output_schema(models, level))
            return _cotransform(
                df,
                X_df,
                self._forecast_series_X,
                params=dict(models=models, freq=freq, 
                            kwargs=kwargs, fallback_model=fallback_model),
                schema=schema,
                partition={"by": "unique_id"},
                engine=self._engine,
                engine_conf=self._conf,
                **self._transform_kwargs,
            )
            

    def cross_validation(
            self, 
            df,
            models,
            freq,
            fallback_model=None,
            **kwargs: Any, 
        ) -> Any:
        """Temporal Cross-Validation with core.StatsForecast and FugueBackend.

        This method uses Fugue's transform function, in combination with 
        `core.StatsForecast`'s cross-validation to efficiently fit a list of StatsForecast 
        models through multiple training windows, in either chained or rolled manner.

        `StatsForecast.models`' speed along with Fugue's distributed computation allow to 
        overcome this evaluation technique high computational costs. Temporal cross-validation 
        provides better model's generalization measurements by increasing the test's length 
        and diversity.

        **Parameters:**<br>
        `df`: pandas.DataFrame, with columns [`unique_id`, `ds`, `y`] and exogenous.<br>
        `freq`: str, frequency of the data, [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>
        `models`: List[typing.Any], list of instantiated objects `StatsForecast.models`.<br>
        `fallback_model`: Any, Model to be used if a model fails.<br>

        **Returns:**<br>
        `fcsts_df`: pandas.DataFrame, with `models` columns for point predictions and probabilistic
        predictions for all fitted `models`.<br>
        
        **References:**<br>
        The [core.StatsForecast's cross validation](https://nixtla.github.io/statsforecast/core.html#statsforecast.cross_validation)
        method documentation.<br>
        [Rob J. Hyndman and George Athanasopoulos (2018). "Forecasting principles and practice, Temporal Cross-Validation"](https://otexts.com/fpp3/tscv.html).
        """
        level = kwargs.get("level", [])
        schema = "*-y+" + str(self._get_output_schema(models, level, mode="cv"))
        return transform(
            df,
            self._cv,
            params=dict(models=models, freq=freq, 
                        kwargs=kwargs, 
                        fallback_model=fallback_model),
            schema=schema,
            partition={"by": "unique_id"},
            engine=self._engine,
            engine_conf=self._conf,
            **self._transform_kwargs,
        )

    def _forecast_series(self, df: pd.DataFrame, models, freq, fallback_model, kwargs) -> pd.DataFrame:
        model = _StatsForecast(df=df, models=models, freq=freq, 
                               fallback_model=fallback_model, n_jobs=1)
        return model.forecast(**kwargs).reset_index()
    
    # schema: unique_id:str, ds:str, *
    def _forecast_series_X(self, df: pd.DataFrame, X_df: pd.DataFrame, models, freq, fallback_model, kwargs) -> pd.DataFrame:
        model = _StatsForecast(df=df, models=models, freq=freq, 
                               fallback_model=fallback_model, n_jobs=1)
        if len(X_df) != kwargs['h']:
            raise Exception(
                'Please be sure that your exogenous variables `X_df` '
                'have the same length than your forecast horizon `h`'
            )
        return model.forecast(X_df=X_df, **kwargs).reset_index()

    def _cv(self, df: pd.DataFrame, models, freq, fallback_model, kwargs) -> pd.DataFrame:
        model = _StatsForecast(df=df, models=models, freq=freq, 
                               fallback_model=fallback_model, n_jobs=1)
        return model.cross_validation(**kwargs).reset_index()

    def _get_output_schema(self, models, level=None, mode="forecast") -> Schema:
        cols: List[Any] = []
        if level is None:
            level = []
        for model in models:
            has_levels = (
                "level" in inspect.signature(getattr(model, "forecast")).parameters
                and len(level) > 0
            )
            cols.append((repr(model), np.float32))
            if has_levels:
                cols.extend([(f"{repr(model)}-lo-{l}", np.float32) for l in reversed(level)])
                cols.extend([(f"{repr(model)}-hi-{l}", np.float32) for l in level])
        if mode == "cv":
            cols = [("cutoff", "datetime"), ("y", np.float32)] + cols
        return Schema(cols)
    

@make_backend.candidate(lambda obj, *args, **kwargs: isinstance(obj, ExecutionEngine))
def _make_fugue_backend(obj:ExecutionEngine, *args, **kwargs) -> ParallelBackend:
    return FugueBackend(obj, **kwargs)

In [ ]:
from statsforecast.core import StatsForecast
from statsforecast.models import ( 
    AutoARIMA,
    AutoETS,
)
from statsforecast.utils import generate_series

n_series = 4
horizon = 7

series = generate_series(n_series)

sf = StatsForecast(
    models=[AutoETS(season_length=7)],
    freq='D',
)

sf.cross_validation(df=series, h=horizon, step_size = 24,
    n_windows = 2, level=[90]).head()

,ds,cutoff,y,AutoETS,AutoETS-lo-90,AutoETS-hi-90
unique_id,,,,,,
0,2000-07-10,2000-07-09,2.472186,2.264802,2.029021,2.500583
0,2000-07-11,2000-07-09,3.369775,3.207784,2.972003,3.443565
0,2000-07-12,2000-07-09,4.245229,4.248131,4.012350,4.483912
0,2000-07-13,2000-07-09,5.113708,5.267366,5.031586,5.503148
0,2000-07-14,2000-07-09,6.127178,6.203136,5.967356,6.438918


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Make unique_id a column
series = series.reset_index()
series['unique_id'] = series['unique_id'].astype(str)

# Convert to Spark
sdf = spark.createDataFrame(series)

# Returns a Spark DataFrame
sf.cross_validation(df=sdf, h=horizon, step_size = 24,
    n_windows = 2, level=[90]).show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/08 23:54:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/08 23:54:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---------+-------------------+-------------------+-----------+----------+-------------+-------------+
|unique_id|                 ds|             cutoff|          y|   AutoETS|AutoETS-lo-90|AutoETS-hi-90|
+---------+-------------------+-------------------+-----------+----------+-------------+-------------+
|        1|2000-03-07 00:00:00|2000-03-06 00:00:00|  1.1842923| 1.1227854|   0.88283557|    1.3627354|
|        1|2000-03-08 00:00:00|2000-03-06 00:00:00|  2.0684502| 2.3335178|     2.093568|    2.5734677|
|        1|2000-03-09 00:00:00|2000-03-06 00:00:00|   3.411059|  3.249278|    3.0093281|    3.4892278|
|        1|2000-03-10 00:00:00|2000-03-06 00:00:00|   4.094924| 4.3513813|    4.1114316|    4.5913315|
|        1|2000-03-11 00:00:00|2000-03-06 00:00:00|  5.2556596| 5.2070827|     4.967133|     5.447033|
|        1|2000-03-12 00:00:00|2000-03-06 00:00:00|  6.1121583|   6.28834|      6.04839|      6.52829|
|        1|2000-03-13 00:00:00|2000-03-06 00:00:00| 0.04892224|0.25212684

In [ ]:
show_doc(FugueBackend, title_level=3)

---

[source](https://github.com/Nixtla/statsforecast/blob/main/statsforecast/distributed/fugue.py#L48){target="_blank" style="float:right; font-size:smaller"}

### FugueBackend

>      FugueBackend (engine:Any=None, conf:Any=None, **transform_kwargs:Any)

FugueBackend for Distributed Computation.
[Source code](https://github.com/Nixtla/statsforecast/blob/main/statsforecast/distributed/fugue.py).

This class uses [Fugue](https://github.com/fugue-project/fugue) backend capable of distributing 
computation on Spark, Dask and Ray without any rewrites.

**Parameters:**<br>
`engine`: fugue.ExecutionEngine, a selection between Spark, Dask, and Ray.<br>
`conf`: fugue.Config, engine configuration.<br>
`**transform_kwargs`: additional kwargs for Fugue's transform method.<br>

**Notes:**<br>
A short introduction to Fugue, with examples on how to scale pandas code to Spark, Dask or Ray
 is available [here](https://fugue-tutorials.readthedocs.io/tutorials/quick_look/ten_minutes.html).

## Dask Distributed Predictions

Here we provide an example for the distribution of the `StatsForecast` predictions using `Fugue` to execute the code in a Dask cluster.

To do it we instantiate the `FugueBackend` class with a `DaskExecutionEngine`.

In [ ]:
#| eval: false
import dask.dataframe as dd
from dask.distributed import Client
from fugue_dask import DaskExecutionEngine
from statsforecast import StatsForecast
from statsforecast.models import Naive
from statsforecast.utils import generate_series

# Generate Synthetic Panel Data
df = generate_series(10).reset_index()
df['unique_id'] = df['unique_id'].astype(str)
df = dd.from_pandas(df, npartitions=10)

# Instantiate FugueBackend with DaskExecutionEngine
dask_client = Client()
engine = DaskExecutionEngine(dask_client=dask_client)

/opt/anaconda3/envs/nixtla/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51914 instead
  warnings.warn(


We have simply create the class to the usual `StatsForecast` instantiation.

In [ ]:
#| eval: false
sf = StatsForecast(models=[Naive()], freq='D')

### Distributed Forecast

For extremely fast distributed predictions we use FugueBackend as backend that operates like the original [StatsForecast.forecast](https://nixtla.github.io/statsforecast/core.html#statsforecast.forecast) method.

It receives as input a pandas.DataFrame with columns [`unique_id`,`ds`,`y`] and exogenous, where the `ds` (datestamp) column should be of a format expected by Pandas. The `y` column must be numeric, and represents the measurement we wish to forecast. And the `unique_id` uniquely identifies the series in the panel data.

In [ ]:
#| eval: false

# Distributed predictions with FugueBackend.
sf.forecast(df=df, h=12).compute()

,unique_id,ds,Naive
0,3,2000-08-30,6.182456
1,3,2000-08-31,6.182456
2,3,2000-09-01,6.182456
3,3,2000-09-02,6.182456
4,3,2000-09-03,6.182456
...,...,...,...
7,8,2000-03-07,0.162962
8,8,2000-03-08,0.162962
9,8,2000-03-09,0.162962
10,8,2000-03-10,0.162962


In [ ]:
#| hide
#| eval: false
# fallback model
class FailNaive:
    def forecast(self):
        pass
    def __repr__(self):
        return 'Naive'
sf = StatsForecast(models=[Naive()], freq='D', fallback_model=Naive())
dask_fcst = sf.forecast(df=df, h=12).compute()
fcst_stats = sf.forecast(df=df.compute(), h=12)
test_eq(dask_fcst.sort_values(by=['unique_id', 'ds']).reset_index(drop=True).astype({"unique_id": str}), 
        fcst_stats.reset_index().astype({"unique_id": str}))

In [ ]:
#| hide
#| eval: false

# Distributed exogenous regressors
class ReturnX:
    
    def __init__(self):
        pass
    
    def fit(self, y, X):
        return self
    
    def predict(self, h, X):
        mean = X
        return X
    
    def __repr__(self):
        return 'ReturnX'
    
    def forecast(self, y, h, X=None, X_future=None, fitted=False):
        return {'mean': X_future.flatten()}
    
    def new(self):
        b = type(self).__new__(type(self))
        b.__dict__.update(self.__dict__)
        return b
    
df_w_ex = pd.DataFrame(
    {
        'ds': np.hstack([np.arange(10), np.arange(10)]),
        'y': np.random.rand(20),
        'x': np.arange(20, dtype=np.float32),
    },
    index=pd.Index([0] * 10 + [1] * 10, name='unique_id'),
).reset_index()
train_mask = df_w_ex['ds'] < 6
train_df = dd.from_pandas(df_w_ex[train_mask], npartitions=10)
test_df = df_w_ex[~train_mask]
xreg = dd.from_pandas(test_df.drop(columns='y').reset_index(drop=True), npartitions=10)

In [ ]:
#| hide
#| eval: false

# Distributed exogenous regressors
fcst_x = StatsForecast(models=[ReturnX()], freq='D')
res = fcst_x.forecast(df=train_df, 
                      X_df=xreg, 
                      h=4).compute()
expected_res = xreg.rename(columns={'x': 'ReturnX'}).compute()
# we expect strings for unique_id, and ds using exogenous
expected_res[['unique_id', 'ds']] = expected_res[['unique_id', 'ds']].astype(str)
pd.testing.assert_frame_equal(res.sort_values('unique_id').reset_index(drop=True), 
                              expected_res, 
                              check_dtype=False, 
                              check_index_type=False)

### Distributed Cross-Validation

For extremely fast distributed temporcal cross-validation we use `cross_validation` method that operates like the original [StatsForecast.cross_validation](https://nixtla.github.io/statsforecast/core.html#statsforecast) method.

In [ ]:
#| eval: false

# Distributed cross-validation with FugueBackend.
sf.cross_validation(df=df, h=12, n_windows=2).compute()

,unique_id,ds,cutoff,y,Naive
0,3,2000-08-17,2000-08-16,1.375511,0.390740
1,3,2000-08-18,2000-08-16,2.463606,0.390740
2,3,2000-08-19,2000-08-16,3.014476,0.390740
3,3,2000-08-20,2000-08-16,4.447845,0.390740
4,3,2000-08-21,2000-08-16,5.196284,0.390740
...,...,...,...,...,...
19,8,2000-02-24,2000-02-16,3.267941,2.483478
20,8,2000-02-25,2000-02-16,4.333817,2.483478
21,8,2000-02-26,2000-02-16,5.410231,2.483478
22,8,2000-02-27,2000-02-16,6.115387,2.483478


In [ ]:
#| hide
#| eval: false
from statsforecast.models import Naive
from statsforecast.utils import generate_series

# Generate Synthetic Panel Data.
df = generate_series(10).reset_index()
df['unique_id'] = df['unique_id'].astype(str)
df = dd.from_pandas(df, npartitions=10)

# Distribute predictions.
sf = StatsForecast(models=[Naive()], freq='D')
fcst_fugue = sf.forecast(df=df, h=12).compute().sort_values(['unique_id', 'ds']).reset_index(drop=True)
fcst_stats = sf.forecast(df=df.compute(), h=12).reset_index().astype({"unique_id": str})
test_eq(fcst_fugue, fcst_stats)

# Distribute cross-validation predictions.
sf = StatsForecast(models=[Naive()], freq='D')
fcst_fugue = sf.cross_validation(df=df, h=12).compute().sort_values(['unique_id', 'ds', 'cutoff']).reset_index(drop=True)
fcst_stats = sf.cross_validation(df=df.compute(), h=12).reset_index().astype({"unique_id": str})
test_eq(fcst_fugue, fcst_stats)

# fallback model
class FailNaive:
    def forecast(self):
        pass
    def __repr__(self):
        return 'Naive'
    
#cross validation fallback model
fcst = StatsForecast(models=[FailNaive()], freq='D', fallback_model=Naive())
fcst_fugue = fcst.cross_validation(df=df, h=12).compute().sort_values(['unique_id', 'ds', 'cutoff']).reset_index(drop=True)
fcst_stats = sf.cross_validation(df=df.compute(), h=12).reset_index().astype({"unique_id": str})
test_eq(fcst_fugue, fcst_stats)

In [ ]:
#| hide
#| eval: false
# test ray integration
import ray
from statsforecast.models import Naive
from statsforecast.utils import generate_series

# Generate Synthetic Panel Data.
df = generate_series(10).reset_index()
df['unique_id'] = df['unique_id'].astype(str)
df = ray.data.from_pandas(df).repartition(2)

# Distribute predictions.
sf = StatsForecast(models=[Naive()], freq='D')
fcst_fugue = sf.forecast(df=df, h=12).to_pandas().sort_values(['unique_id', 'ds']).reset_index(drop=True)
fcst_stats = sf.forecast(df=df.to_pandas(), h=12).reset_index().astype({"unique_id": str})
test_eq(fcst_fugue, fcst_stats)

# Distribute cross-validation predictions.
fcst = StatsForecast(models=[Naive()], freq='D')
fcst_fugue = fcst.cross_validation(df=df, h=12).to_pandas().sort_values(['unique_id', 'ds', 'cutoff']).reset_index(drop=True)
fcst_stats = sf.cross_validation(df=df.to_pandas(), h=12).reset_index().astype({"unique_id": str})
test_eq(fcst_fugue, fcst_stats)

# fallback model
class FailNaive:
    def forecast(self):
        pass
    def __repr__(self):
        return 'Naive'
    
#cross validation fallback model
sf = StatsForecast(models=[FailNaive()], freq='D', fallback_model=Naive())
fcst_fugue = sf.cross_validation(df=df, h=12).to_pandas().sort_values(['unique_id', 'ds', 'cutoff']).reset_index(drop=True)
fcst_stats = sf.cross_validation(df=df.to_pandas(), h=12).reset_index().astype({"unique_id": str})
test_eq(fcst_fugue, fcst_stats)

2023-07-08 23:56:08,907	INFO worker.py:1625 -- Started a local Ray instance.
2023-07-08 23:56:12,762	WARNING dataset.py:4066 -- Deprecation warning: use Dataset.materialize() instead of fully_executed().
2023-07-08 23:56:12,766	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[Repartition]
2023-07-08 23:56:12,769	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running: 0.0/8.0 CPU, 0.0/0.0 GPU, 0.0 MiB/512.0 MiB object_store_memory 0:   0%|          | 0/2 [00:00<?, ?it/s]
                                                                                                                          
2023-07-08 23:56:13,645	WARNING dataset.py:4066 -- Deprecation warning: use Dataset.materialize() instead of fully_executed().
2023-07-08 23:56:13,666	WA

In [ ]:
#| hide
#| eval: false

# Distributed exogenous regressors
sf = StatsForecast(models=[ReturnX()], freq='D')
res = sf.forecast(df=train_df, 
                  X_df=xreg, 
                  h=4).compute()
expected_res = xreg.compute().rename(columns={'x': 'ReturnX'})
# we expect strings for unique_id, and ds using exogenous
expected_res[['unique_id', 'ds']] = expected_res[['unique_id', 'ds']].astype(str)
pd.testing.assert_frame_equal(res.sort_values('unique_id').reset_index(drop=True), 
                              expected_res, 
                              check_dtype=False, 
                              check_index_type=False)